In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['imdb_master.csv']


In [2]:
import nltk
from collections import Counter
import itertools

In [3]:
import torch

Рассмотрим преобразование текстов в удобоваримый для нейронной сети вид.<br>
А именно:<br>
    - Текст разбивается на слова (токенизация, знаки препинания считаются словами)<br>
    - Слова подсчитываются для формирования ограниченного словаря. Каждому слову сопоставляется определеннный номер в словаре. 
    Редким словам назначается специальный номер (эквивалентно замене редких слов на спец. слово <UNK> (неизвестное слово)). 
    - Последовательности слов преобразуются в последовательности номеров слов. Добавляются спец. слова для обозначения начала и конца текста. 
    - Полученные последовательности выравниваются по заданной максимальной длине через обрезание или дополнение номером спец.символа <PAD>


Класс для хранения текста в виде последовательности номеров и его закодированной метки

In [4]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, label_id):
        self.input_ids = input_ids
        self.label_id = label_id

Класс словаря. Метод word2id возвращает номер слова, id2word - наоборот, восстанавливает слово.

In [5]:
class Vocab:
    def __init__(self, itos, unk_index):
        self._itos = itos
        self._stoi = {word:i for i, word in enumerate(itos)}
        self._unk_index = unk_index
        
    def __len__(self):
        return len(self._itos)
    
    def word2id(self, word):
        idx = self._stoi.get(word)
        if idx is not None:
            return idx
        return self._unk_index
    
    def id2word(self, idx):
        return self._itos[idx]

In [6]:
from tqdm import tqdm_notebook

Интерфейс объекта, преобразующего тексты в последовательности номеров.
transform выполняет преобразование при помощи словаря.
fit_transform выучивает словарь из текста и возвращает такое же преобразование при помощи свежеполученного словаря.

In [7]:
class TextToIdsTransformer:
    def transform():
        raise NotImplementedError()
        
    def fit_transform():
        raise NotImplementedError()



Простая реализация данного интерфейса. Разбиение на слова производится с помощью библиотеки NLTK.
В словаре содержатся несколько спец. слов.
После токенизации, к полученной последовательности слов добавляются слева и справа спец. слова для начала и конца текста.

In [8]:
class SimpleTextTransformer(TextToIdsTransformer):
    def __init__(self, max_vocab_size):
        self.special_words = ['<PAD>', '</UNK>', '<S>', '</S>']
        self.unk_index = 1
        self.pad_index = 0
        self.vocab = None
        self.max_vocab_size = max_vocab_size
        
    def tokenize(self, text):
        return nltk.tokenize.word_tokenize(text.lower())
        
    def build_vocab(self, tokens):
        itos = []
        itos.extend(self.special_words)
        
        token_counts = Counter(tokens)
        for word, _ in token_counts.most_common(self.max_vocab_size - len(self.special_words)):
            itos.append(word)
            
        self.vocab = Vocab(itos, self.unk_index)
    
    def transform(self, texts):
        result = []
        for text in texts:
            tokens = ['<S>'] + self.tokenize(text) + ['</S>']
            ids = [self.vocab.word2id(token) for token in tokens]
            result.append(ids)
        return result
    
    def fit_transform(self, texts):
        result = []
        tokenized_texts = [self.tokenize(text) for text in texts]
        self.build_vocab(itertools.chain(*tokenized_texts))
        for tokens in tokenized_texts:
            tokens = ['<S>'] + tokens + ['</S>']
            ids = [self.vocab.word2id(token) for token in tokens]
            result.append(ids)
        return result

Строим экземпляр входных данных. Обеспечиваем длину последовательности номеров равной max_seq_len. 

In [9]:
def build_features(token_ids, label, max_seq_len, pad_index, label_encoding):
    if len(token_ids) >= max_seq_len:
        ids = token_ids[:max_seq_len]
    else:
        ids = token_ids + [pad_index for _ in range(max_seq_len - len(token_ids))]
    return InputFeatures(ids, label_encoding[label])
        

Собираем экземпляры в тензоры

In [21]:
def features_to_tensor(list_of_features):
    text_tensor = torch.tensor([example.input_ids for example in list_of_features], dtype=torch.long)
    labels_tensor = torch.tensor([example.label_id for example in list_of_features], dtype=torch.long)
    return text_tensor, labels_tensor

In [11]:
from sklearn import model_selection

In [12]:
imdb_df = pd.read_csv('../input/imdb_master.csv', encoding='latin-1')
dev_df = imdb_df[(imdb_df.type == 'train') & (imdb_df.label != 'unsup')]
test_df = imdb_df[(imdb_df.type == 'test')]
train_df, val_df = model_selection.train_test_split(dev_df, test_size=0.05, stratify=dev_df.label)

In [13]:
max_seq_len=200
classes = {'neg': 0, 'pos' : 1}

In [14]:
text2id = SimpleTextTransformer(10000)

train_ids = text2id.fit_transform(train_df['review'])
val_ids = text2id.transform(val_df['review'])
test_ids = text2id.transform(test_df['review'])

In [15]:
print(train_df.review.iloc[0][:160])
print(train_ids[0][:30])

This is one of my all time favorite movies and I would recommend it to anyone. On my list of favorite movies (mental list, mind) the only ones on par with it ar
[2, 19, 11, 42, 9, 72, 45, 78, 507, 120, 7, 18, 70, 387, 16, 10, 263, 6, 32, 72, 1063, 9, 507, 120, 30, 1709, 1063, 5, 360, 29]


In [16]:
train_features = [build_features(token_ids, label,max_seq_len, text2id.pad_index, classes) 
                  for token_ids, label in zip(train_ids, train_df['label'])]

val_features = [build_features(token_ids, label,max_seq_len, text2id.pad_index, classes) 
                  for token_ids, label in zip(val_ids, val_df['label'])]

test_features = [build_features(token_ids, label,max_seq_len, text2id.pad_index, classes) 
                  for token_ids, label in zip(test_ids, test_df['label'])]

In [17]:
print(train_features[3].input_ids)

[2, 45, 19, 1, 435, 9, 28, 2018, 718, 11, 1561, 1, 6404, 74, 189, 1, 1587, 6, 778, 46, 198, 1135, 1, 5, 2521, 1, 5, 310, 1, 5, 7, 851, 1, 313, 32, 8, 299, 7, 96, 33, 97, 741, 729, 1984, 5, 59, 2689, 1, 5, 9, 48, 755, 26, 6, 444, 89, 19, 170, 2810, 6, 54, 11, 33, 42, 216, 383, 5, 218, 5, 337, 5, 56, 952, 50, 6405, 636, 702, 10, 6405, 552, 6, 4, 2430, 5, 1, 2521, 1, 9, 729, 765, 1399, 92, 371, 6, 1135, 1, 92, 371, 6, 310, 1, 92, 371, 6, 4, 215, 3757, 92, 371, 6, 61, 19, 11, 8, 648, 26, 5, 1, 1354, 11, 8, 1, 668, 6, 14, 15, 12, 13, 14, 15, 12, 13, 623, 92, 189, 105, 26, 1324, 111, 217, 1786, 9, 111, 8797, 6, 609, 16, 21, 17, 4, 636, 912, 92, 35, 437, 5, 539, 5, 7, 1117, 47, 34, 42, 241, 6, 56, 44, 4, 69, 2187, 9, 4, 235, 11, 64, 8165, 51, 9994, 20, 31, 140, 54, 21, 42, 7, 79, 42, 291, 19, 2867, 198, 113, 6, 147, 5, 31, 58, 33, 181, 62]


In [22]:
train_tensor, train_labels = features_to_tensor(train_features)
val_tensor, val_labels = features_to_tensor(val_features)
test_tensor, test_labels = features_to_tensor(test_features)

In [23]:
print(train_tensor.size())

torch.Size([23750, 200])


In [ ]:
print(len(text2id.vocab))